In [6]:

# Import required libraries
import chess
import chess.engine
import chess.pgn
import subprocess
import os
import json
import csv
from IPython.display import display

# Paths to your engines and Maia models
stockfish_path = "/opt/homebrew/bin/stockfish"
maia_model_paths = {
    1100: "maia_weights/maia-1100.pb.gz",
    1200: "maia_weights/maia-1200.pb.gz",
    1300: "maia_weights/maia-1300.pb.gz",
    1400: "maia_weights/maia-1400.pb.gz",
    1500: "maia_weights/maia-1500.pb.gz",
    1600: "maia_weights/maia-1600.pb.gz",
    1700: "maia_weights/maia-1700.pb.gz",
    1800: "maia_weights/maia-1800.pb.gz",
    1900: "maia_weights/maia-1900.pb.gz",
}

# Initialize Stockfish engine
print("Initializing Stockfish engine...")
stockfish_engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

# Initialize Maia engines
print("Initializing Maia engines...")
maia_engines = {}
for elo, path in maia_model_paths.items():
    maia_engines[elo] = chess.engine.SimpleEngine.popen_uci(["lc0", f"--weights={path}"])

Initializing Stockfish engine...


<UciProtocol (pid=18472)>: stderr >>        _
<UciProtocol (pid=18472)>: stderr >> |   _ | |
<UciProtocol (pid=18472)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18473)>: stderr >>        _
<UciProtocol (pid=18473)>: stderr >> |   _ | |
<UciProtocol (pid=18473)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18474)>: stderr >>        _
<UciProtocol (pid=18474)>: stderr >> |   _ | |
<UciProtocol (pid=18474)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18475)>: stderr >>        _
<UciProtocol (pid=18475)>: stderr >> |   _ | |
<UciProtocol (pid=18475)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18476)>: stderr >>        _
<UciProtocol (pid=18476)>: stderr >> |   _ | |
<UciProtocol (pid=18476)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18477)>: stderr >>        _
<UciProtocol (pid=18477)>: stderr >> |   _ | |
<UciProtocol (pid=18477)>: 

Initializing Maia engines...


In [7]:
# Part 2: Analysis Functions

import chess.pgn

# Function to calculate average error for a position using Stockfish
def calculate_average_error(board, stockfish_engine):
    legal_moves = list(board.legal_moves)
    total_error = 0
    num_moves = len(legal_moves)

    for move in legal_moves:
        board.push(move)
        info = stockfish_engine.analyse(board, chess.engine.Limit(time=0.1))
        eval_score = info["score"].relative.score(mate_score=10000) / 100.0
        total_error += abs(eval_score)
        board.pop()

    average_error = total_error / num_moves if num_moves > 0 else 0
    return average_error

# Function to find classic puzzles in a game
def find_classic_puzzles(game, stockfish_engine, threshold=200):
    board = game.board()
    puzzles = []

    for move in game.mainline_moves():
        fen_before = board.fen()

        # Analyze the position before the move
        info = stockfish_engine.analyse(board, chess.engine.Limit(time=0.1), multipv=2)
        if len(info) < 2:
            board.push(move)
            continue

        best_move = info[0]["pv"][0]
        best_eval = info[0]["score"].relative.score(mate_score=10000)
        second_best_eval = info[1]["score"].relative.score(mate_score=10000)

        # Check if the difference is large enough and the best move is not a simple recapture
        if abs(best_eval - second_best_eval) >= threshold and not board.is_capture(best_move):
            puzzles.append((fen_before, best_move, best_eval, second_best_eval))
            print(f"Puzzle found: {fen_before}, Best move: {best_move}, Eval difference: {abs(best_eval - second_best_eval)}")

        board.push(move)

    return puzzles

# Function to evaluate Maia easiness
def evaluate_maia_easiness(fen, best_move, maia_engines):
    board = chess.Board(fen)
    easiness_scores = {}

    for elo, engine in maia_engines.items():
        result = engine.analyse(board, chess.engine.Limit(nodes=1))
        if "pv" in result and best_move in result["pv"]:
            easiness_scores[elo] = result["pv"].index(best_move) / len(result["pv"])

    return easiness_scores


In [8]:
# Part 3: Analyzing Games and Saving Results

import csv
import json

# Function to save results incrementally to CSV
def save_results_to_csv(csv_file_path, data):
    with open(csv_file_path, "a", newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        for fen, best_move, best_eval, second_best_eval, maia_easiness in data:
            csvwriter.writerow([
                fen,
                best_move.uci(),  # Convert Move object to UCI notation
                best_eval,
                second_best_eval,
                maia_easiness
            ])

# Function to save results incrementally to JSON
def save_results_to_json(json_file_path, data):
    if os.path.exists(json_file_path):
        with open(json_file_path, "r") as jsonfile:
            json_data = json.load(jsonfile)
    else:
        json_data = []

    for fen, best_move, best_eval, second_best_eval, maia_easiness in data:
        json_data.append({
            "fen": fen,
            "best_move": best_move.uci(),  # Convert Move object to UCI notation
            "best_eval": best_eval,
            "second_best_eval": second_best_eval,
            "maia_easiness": maia_easiness
        })

    with open(json_file_path, "w") as jsonfile:
        json.dump(json_data, jsonfile, indent=4)

# Function to sort and save results
def sort_and_save_results(puzzles, output_dir="results/classic_puzzles"):
    os.makedirs(output_dir, exist_ok=True)
    puzzles.sort(key=lambda x: x["maia_easiness"], reverse=True)

    csv_file_path = os.path.join(output_dir, "classic_puzzles.csv")
    json_file_path = os.path.join(output_dir, "classic_puzzles.json")

    with open(csv_file_path, "w", newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["FEN", "Best Move", "Best Eval", "Second Best Eval", "Maia Easiness"])
        for puzzle in puzzles:
            csvwriter.writerow([puzzle["fen"], puzzle["best_move"], puzzle["best_eval"], puzzle["second_best_eval"], puzzle["maia_easiness"]])

    with open(json_file_path, "w") as jsonfile:
        json.dump(puzzles, jsonfile, indent=4)

# Load PGN files and find classic puzzles
pgn_directory = "analysis_pgns_test"
all_puzzles = []

print("Starting to find classic puzzles in PGN files...")

for pgn_file in os.listdir(pgn_directory):
    if pgn_file.endswith(".pgn"):
        pgn_file_path = os.path.join(pgn_directory, pgn_file)
        print(f"Analyzing {pgn_file_path}...")
        all_games = load_games_from_pgn(pgn_file_path)

        for game in all_games:
            print(f"Analyzing game {game.headers.get('Event', 'Unknown Event')}")
            puzzles = find_classic_puzzles(game, stockfish_engine)
            for fen, best_move, best_eval, second_best_eval in puzzles:
                maia_easiness = evaluate_maia_easiness(fen, best_move, maia_engines)
                all_puzzles.append((fen, best_move, best_eval, second_best_eval, maia_easiness))

            # Save results incrementally
            save_results_to_csv("classic_puzzles.csv", all_puzzles)
            save_results_to_json("classic_puzzles.json", all_puzzles)

print("Classic puzzles analysis complete. Results saved.")


Starting to find classic puzzles in PGN files...
Analyzing analysis_pgns_test/QFkiT2sJ---1890 Steinitz vs. Gunsburg.pgn...
Loaded 19 games from PGN file analysis_pgns_test/QFkiT2sJ---1890 Steinitz vs. Gunsburg.pgn.
Analyzing game World Championship Match 1890/91
Analyzing game World Championship Match 1890/91
Analyzing game World Championship Match 1890/91
Analyzing game World Championship Match 1890/91
Puzzle found: 6k1/1R6/4Np2/5rpp/3PN3/6KP/8/8 b - - 1 53, Best move: g8h8, Eval difference: 9281
Puzzle found: 7k/6R1/4Np2/5rp1/3PN2p/7P/6K1/8 b - - 1 55, Best move: g5g4, Eval difference: 230
Analyzing game World Championship Match 1890/91
Puzzle found: 2kr1b1r/pp1n1ppp/2p5/5b2/4P1n1/1KN2N2/PP2BPPP/R1B2R2 b - - 0 14, Best move: d7c5, Eval difference: 223
Puzzle found: 2kr3r/pp1nb1pp/2p2p2/5PP1/4b1BN/1K6/PP5P/R1B2R2 b - - 1 24, Best move: d7c5, Eval difference: 234
Puzzle found: 2kr3r/pp2b1pp/2p2p2/2n2PP1/4b1BN/K7/PP5P/R1B2R2 b - - 3 25, Best move: d8d3, Eval difference: 9392
Puzzle foun

FileNotFoundError: [Errno 2] No such file or directory: 'analysis_pgns_test/wgJ634pR---1984 Karpov vs. Kasparov.pgn'